In [ ]:
import math
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
import torchvision
#
import einops
from torchvision import datasets
import torchvision.transforms as T
import matplotlib.pyplot as plt
import random

In [ ]:
xx = np.linspace(-0.5, 1.5, 51)

t_freq = 3
t_symm = 0.5
#
yy = (1 - np.tanh(t_freq * (xx - t_symm))) * 0.5
plt.plot(xx, yy)
plt.axvline(t_symm)
plt.axvline(0)
plt.axvline(1)

In [ ]:
xx = np.linspace(-0.5, 1.5, 51)

t_freq = 3
t_symm = 0.5
#
yy = (1 - np.tanh(t_freq * (xx - t_symm))) * 0.5
plt.plot(xx, yy)
plt.axvline(t_symm)
plt.axvline(0)
plt.axvline(1)

# POS. Embedding 4D

In [ ]:
h, w = 28, 28
d = 4
#
pos_w = torch.linspace(0, 1, w)
pos_h = torch.linspace(0, 1, h)

# linear
pe = torch.zeros(4, h, w)
pe[0] = torch.linspace(0, 1, w).unsqueeze(1).repeat(1, h)
pe[1] = torch.linspace(1, 0, w).unsqueeze(1).repeat(1, h)
pe[2] = torch.linspace(0, 1, h).T.repeat(w, 1)
pe[3] = torch.linspace(1, 0, h).T.repeat(w, 1)

#
# exponential -> not symmetric
pe = torch.zeros(4, h, w)
l = -310
pe[0] = torch.exp(torch.linspace(0, 1, w) * -l).unsqueeze(1).repeat(1, h)
pe[1] = torch.exp(torch.linspace(1, 0, w) * -l).unsqueeze(1).repeat(1, h)
pe[2] = torch.exp(torch.linspace(0, 1, h) * -l).T.repeat(w, 1)
pe[3] = torch.exp(torch.linspace(1, 0, h) * -l).T.repeat(w, 1)


# tanh
t_freq = 2
t_symm = 0.5
pe[0] = (1 - torch.tanh(t_freq * (torch.linspace(0, 1, w) - t_symm)).unsqueeze(1).repeat(1, h)) * 0.5
pe[1] = (1 - torch.tanh(t_freq * (torch.linspace(1, 0, w) - t_symm)).unsqueeze(1).repeat(1, h)) * 0.5
pe[2] = (1 - torch.tanh(t_freq * (torch.linspace(0, 1, h) - t_symm)).T.repeat(w, 1)) * 0.5
pe[3] = (1 - torch.tanh(t_freq * (torch.linspace(1, 0, h) - t_symm)).T.repeat(w, 1)) * 0.5

In [ ]:
for ipe in pe:
    plt.imshow(ipe, cmap="gray")
    plt.show()

In [ ]:
E = pe.permute(1,2,0).reshape(h*w,d)

In [ ]:
G = []
for i in range(h*w):
    for j in range(h*w):
        a = E[i]
        b = E[j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(h,w,h,w)
print(S.min(), S.max(), S.shape)

### plot all

In [ ]:
fig, axes = plt.subplots(h,w, figsize=(15, 15))
for xi in range(w):
    for yi in range(h):
        ax = axes[xi][yi]
        ax.imshow(S[xi][yi], cmap="gray")
        ax.axis('off')
plt.show()

In [ ]:
pe.sum(dim=0)

In [ ]:
def pos_tanh_embedding(h, w, t_freq = 2, t_symm = 0.5):
    pe = torch.zeros(4, h, w)
    pe[0] = (1 - torch.tanh(t_freq * (torch.linspace(0, 1, w) - t_symm)).unsqueeze(1).repeat(1, h)) * 0.5
    pe[1] = (1 - torch.tanh(t_freq * (torch.linspace(1, 0, w) - t_symm)).unsqueeze(1).repeat(1, h)) * 0.5
    pe[2] = (1 - torch.tanh(t_freq * (torch.linspace(0, 1, h) - t_symm)).T.repeat(w, 1)) * 0.5
    pe[3] = (1 - torch.tanh(t_freq * (torch.linspace(1, 0, h) - t_symm)).T.repeat(w, 1)) * 0.5
    return pe

### emb on MNIST

In [ ]:
ds_train = datasets.MNIST(
    root = '/mnt/data/pytorch',
    train = True,                         
    transform = T.ToTensor(), 
    download = True,            
)
ds_test = datasets.MNIST(
    root = '/mnt/data/pytorch',
    train = False, 
    transform = T.ToTensor()
)

In [ ]:
x,_ = ds_train[0]
img = x[0][20:27,2:9]
print(img.shape)
plt.imshow(img, cmap="gray", vmin=0, vmax=1)

In [ ]:
x,_ = ds_train[0]
img = x[0]
print(img.shape)
plt.imshow(img, cmap="gray", vmin=0, vmax=1)

In [ ]:
pe = pos_tanh_embedding(x.shape[1], x.shape[2])
E = torch.cat((x, pe), dim=0)
print(E.shape)

In [ ]:
G = []
E = E.permute(1,2,0).reshape(28*28, -1)
for i in range(h*w):
    for j in range(h*w):
        a = E[i]
        b = E[j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(h,w,h,w)
print(S.min(), S.max(), S.shape)

In [ ]:
fig, axes = plt.subplots(h,w, figsize=(15, 15))
for xi in range(w):
    for yi in range(h):
        ax = axes[xi][yi]
        ax.imshow(S[xi][yi], cmap="gray")
        ax.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(S.permute(0,2,1,3).reshape(28*28, 28*28), cmap="gray")
plt.axis("off")

### emb on CIFAR

In [ ]:
ds = torchvision.datasets.CIFAR10(
    root='/mnt/data/pytorch', train=True, download=True, transform=T.ToTensor())

In [ ]:
x,_ = ds[11]
plt.imshow(x.permute(1,2,0))
x.shape

### emb with position - channels cross product

In [ ]:
pe = pos_tanh_embedding(32, 32)
print(pe.shape, x.shape)

In [ ]:
E = torch.einsum("ijk, ljk -> iljk", pe, x).reshape(3*4,32,32)

In [ ]:
G = []
E = E.permute(1,2,0).reshape(32*32, -1)
for i in range(32*32):
    for j in range(32*32):
        a = E[i]
        b = E[j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(32,32,32,32)
print(S.min(), S.max(), S.shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(S.permute(0,2,1,3).reshape(32*32, 32*32), cmap="gray")
plt.axis("off")

### emb with position CAT

In [ ]:
pe = pos_tanh_embedding(32, 32)
E = torch.cat((x, pe), dim=0)
E.shape

In [ ]:
G = []
E = E.permute(1,2,0).reshape(32*32, -1)
for i in range(32*32):
    for j in range(32*32):
        a = E[i]
        b = E[j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(32,32,32,32)
print(S.min(), S.max(), S.shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(S.permute(0,2,1,3).reshape(32*32, 32*32), cmap="gray")
plt.axis("off")

In [ ]:
32**4

### emb without position

In [ ]:
pe = pos_tanh_embedding(32, 32)
#E = torch.cat((x, pe), dim=0)
E = x
E.shape

In [ ]:
G = []
E = E.permute(1,2,0).reshape(32*32, -1)
for i in range(32*32):
    for j in range(32*32):
        a = E[i]
        b = E[j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(32,32,32,32)
print(S.min(), S.max(), S.shape)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(S.permute(0,2,1,3).reshape(32*32, 32*32), cmap="gray")
plt.axis("off")

In [ ]:
plt.imshow(A[0].reshape(32,32))

In [ ]:
A_np = A.numpy()
A.min(), A.max()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=32).fit(A_np)

In [ ]:
for cc in kmeans.cluster_centers_:
    plt.imshow(cc.reshape(32,32), cmap="gray", vmin=0, vmax=1)
    plt.show()

# plot every x

In [ ]:
pass

# OLD

In [ ]:
D_model = 16
height = h = 10E = pe.permute(1,2,0).reshape(h*w,d)
E.shape
width = w = 10
#

pe = torch.zeros(D_model, height, width)
#
d_model = int(D_model / 2)
div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
pos_w = torch.arange(0., width).unsqueeze(1)
pos_h = torch.arange(0., height).unsqueeze(1)
#
print(pe.shape)
print(div_term)

In [ ]:
pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)

In [ ]:
# visualize
E = pe.reshape((D_model, height*width))
print(E.shape)
#
G = []
for i in range(height*width):
    for j in range(height*width):
        a = E[:, i]
        b = E[:, j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(height,width,height,width)
S.min(), S.max()

In [ ]:
fig, axes = plt.subplots(h,w, figsize=(15, 15))
for xi in range(w):
    for yi in range(h):
        ax = axes[xi][yi]
        ax.imshow(S[xi][yi], vmin=S.min(), vmax=S.max(), cmap="gray")
        ax.axis('off')
plt.show()

# Advanced PE

In [ ]:
d = 4
h = 16
w = 16
#

pe = torch.zeros(d, h, w)
#
pos_w = torch.arange(0., w).unsqueeze(1)
pos_h = torch.arange(0., h).unsqueeze(1)
#

In [ ]:
# d must be devisible by 2
pe[0] = torch.sin(pos_w / 10 / 2).transpose(0, 1).unsqueeze(1).repeat(1, h, 1)
pe[1] = torch.cos(pos_w / 10 / 2).transpose(0, 1).unsqueeze(1).repeat(1, h, 1)


pe[2] = torch.sin(pos_h / 10 / 2).transpose(0, 1).unsqueeze(2).repeat(1, 1, w)
pe[3] = torch.cos(pos_h / 10 / 2).transpose(0, 1).unsqueeze(2).repeat(1, 1, w)

In [ ]:
pe.min(), pe.max()

In [ ]:
for ipe in pe:
    plt.imshow(ipe, cmap="gray")
    plt.show()

In [ ]:
pe.min(), pe.max()

In [ ]:
# visualize
E = pe.reshape((d, h*w))
print(E.shape)
#
G = []
for i in range(h*w):
    for j in range(h*w):
        a = E[:, i]
        b = E[:, j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.a3ppend(cs)
S = torch.Tensor(G).reshape(h,w,h,w)
print(S.min(), S.max())

In [ ]:
fig, axes = plt.subplots(h,w, figsize=(15, 15))
for xi in range(w):
    for yi in range(h):
        ax = axes[xi][yi]
        ax.imshow(S[xi][yi], vmin=S.min(), vmax=S.max(), cmap="gray")
        ax.axis('off')
plt.show()

# 2D Linear Embedding

In [ ]:
xx = np.linspace(0, np.pi * 2, 100)
f = 1
plt.plot(xx, np.sin(f * xx))
plt.show()
plt.plot(xx, np.cos(f * xx))
plt.show()

In [ ]:
h, w = 10, 10
d = 4
#
pos_w = torch.linspace(0, 1, w)
pos_h = torch.linspace(0, 1, h)

# sinoid
pe = torch.zeros(4, h, w)
pe[0] = torch.sin(f * pos_w).unsqueeze(1).repeat(1, h)
pe[1] = torch.cos(f * pos_w).unsqueeze(1).repeat(1, h)
pe[2] = torch.sin(f * pos_h).unsqueeze(1).T.repeat(w, 1)
pe[3] = torch.cos(f * pos_h).unsqueeze(1).T.repeat(w, 1)

# linear
pe = torch.zeros(4, h, w)
pe[0] = torch.linspace(0, 1, w).unsqueeze(1).repeat(1, h)
pe[1] = torch.linspace(1, 0, w).unsqueeze(1).repeat(1, h)
pe[2] = torch.linspace(0, 1, h).T.repeat(w, 1)
pe[3] = torch.linspace(1, 0, h).T.repeat(w, 1)

In [ ]:
for ipe in pe:
    plt.imshow(ipe, cmap="gray")
    plt.show()

In [ ]:
E = pe.permute(1,2,0).reshape(h*w,d)
E.shape

In [ ]:
G = []
for i in range(h*w):
    for j in range(h*w):
        a = E[i]
        b = E[j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(h,w,h,w)
print(S.min(), S.max(), S.shape)

In [ ]:
fig, axes = plt.subplots(h,w, figsize=(15, 15))
for xi in range(w):
    for yi in range(h):
        ax = axes[xi][yi]
        ax.imshow(S[xi][yi], vmin=S.min(), vmax=S.max(), cmap="gray")
        ax.axis('off')
plt.show()

# Better embedding

In [ ]:
x = np.linspace(0, 10, 100)
y = np.sin(x/5)
#
plt.plot(x, y)

In [ ]:
plt.plot(x, y)

In [ ]:
a = np.pi/2 / 10
x = np.pi/2 / 10
np.cos(a *  x)

In [ ]:
x = np.linspace(0, 10, 100)
a = np.pi/ 10 / 2
plt.plot(x, np.cos(a * x))

In [ ]:
# want it to be devisible by 4 such thatpos_h = torch.arange(0., h).unsqueeze(1)
# PE(x, y) =
# [sin(x, w), cos(x, w), sin(y, w), cos(y, w)]
h = 12
w = 12
d = 4
#
assert d % 4 == 0
#
n_freqs = d // 4
freq_low = np.pi / 2 / h
freq_high = 10000*2 * np.pi
#
freqs = np.linspace(freq_low, freq_high, n_freqs)

In [ ]:
lmbdas = [2**i for i in range(-1, n_freqs - 1, 1)]pos_h = torch.arange(0., h).unsqueeze(1)
lmbdas

In [ ]:
x = np.linspace(0, h, 100)
for freq in freqs:
    plt.plot(x, np.sin(freq * x))
    plt.plot(x, np.cos(freq * x))
    plt.show()

In [ ]:
def get_freq(k):
    return 1/(10000**(2*k*0.5))

In [ ]:
def get_pe(h, w, d):
    assert d % 4 == 0
    pe = torch.zeros(d, h, w)
    #.shape
    pos_w = torch.arange(0., w).unsqueeze(1)
    pos_h = torch.arange(0., h).unsqueeze(1)

In [ ]:
pe = torch.zeros(d, h, w)
#.shape
pos_w = torch.arange(0., w).unsqueeze(1)
pos_h = torch.arange(0., h).unsqueeze(1)

In [ ]:
d = 16
h = 8
w = 8
pe = torch.zeros(d, h, w)
#.shape
pos_w = torch.arange(0., w).unsqueeze(1)
pos_h = torch.arange(0., h).unsqueeze(1)
#
pe[0::4,::] = torch.sin(pos_w * torch.tensor(freqs)).transpose(0, 1).unsqueeze(1).repeat(1, h, 1)
pe[1::4,::] = torch.cos(pos_w * torch.tensor(freqs)).transpose(0, 1).unsqueeze(1).repeat(1, h, 1)
pe[2::4,::] = torch.sin(pos_h * torch.tensor(freqs)).transpose(0, 1).unsqueeze(2).repeat(1, 1, w)
pe[3::4,::] = torch.cos(pos_h * torch.tensor(freqs)).transpose(0, 1).unsqueeze(2).repeat(1, 1, w)

In [ ]:
# visualize
E = pe.reshape((d, h*w))
print(E.shape)
#
G = []
for i in range(h*w):
    for j in range(h*w):
        a = E[:, i]
        b = E[:, j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(h,w,h,w)
print(S.min(), S.max(), S.shape)

In [ ]:
fig, axes = plt.subplots(h,w, figsize=(15, 15))
for xi in range(w):
    for yi in range(h):
        ax = axes[xi][yi]
        ax.imshow(S[xi][yi], vmin=S.min(), vmax=S.max(), cmap="gray")
        ax.axis('off')
plt.show()

# ...

In [ ]:
# want it to be devisible by 4 such that
# PE(x, y) =
# [sin(x, w), cos(x, w), sin(y, w), cos(y, w)]
h = 7
w = 7
d = 4
#

In [ ]:
def get_pe(h, w):
    pe = torch.zeros(4, h, w)
    #.shape
    pos_w = torch.arange(0., w) / w
    pos_h = torch.arange(0., h) / h
    #
    pe[0:,:] = torch.sin(pos_w.repeat(h).reshape(h, w))
    pe[1:,:] = torch.cos(pos_w.repeat(h).reshape(h, w))
    #
    pe[2:,:] = torch.sin(pos_h.repeat(w).reshape(w, h).T)
    pe[3:,:] = torch.cos(pos_h.repeat(w).reshape(w, h).T)
    return pe

In [ ]:
pe = torch.zeros(d, h, w)
#.shape
pos_w = torch.arange(0., w) / w
pos_h = torch.arange(0., h) / h

In [ ]:
pe[0:,:] = torch.sin(pos_w.repeat(h).reshape(h, w))
pe[1:,:] = torch.cos(pos_w.repeat(h).reshape(h, w))
#
pe[2:,:] = torch.sin(pos_h.repeat(w).reshape(w, h).T)
pe[3:,:] = torch.cos(pos_h.repeat(w).reshape(w, h).T)

In [ ]:
# visualize
E = pe.reshape((d, h*w))
print(E.shape)
#
G = []
for i in range(h*w):
    for j in range(h*w):
        a = E[:, i]
        b = E[:, j]
        cs = F.cosine_similarity(a, b, dim=0)
        G.append(cs)
S = torch.Tensor(G).reshape(h,w,h,w)
print(S.min(), S.max(), S.shape)

In [ ]:
fig, axes = plt.subplots(h,w, figsize=(15, 15))
for xi in range(w):
    for yi in range(h):
        ax = axes[xi][yi]
        ax.imshow(S[xi][yi], vmin=S.min(), vmax=S.max(), cmap="gray")
        ax.axis('off')
plt.show()

In [ ]:
import torch
import einops
from torchvision import datasets
import torchvision.transforms as T
import matplotlib.pyplot as plt
import random

In [ ]:
ds_train = datasets.MNIST(
    root = '/mnt/data/pytorch',
    train = True,                         
    transform = T.ToTensor(), 
    download = True,            
)
ds_test = datasets.MNIST(
    root = '/mnt/data/pytorch',
    train = False, 
    transform = T.ToTensor()
)

In [ ]:
x,_ = ds_train[0]
img = x[0][20:27,2:9]
print(img.shape)
plt.imshow(img, cmap="gray", vmin=0, vmax=1)

In [ ]:
E = torch.cat((img.unsqueeze(0), pe), dim=0)

In [ ]:
X = E.permute((1,2,0)).reshape(7*7,-1)

In [ ]:
A = (X @ X.T).reshape((7,7,7,7))

In [ ]:
x,_ = ds_train[0]
img = x[0][20:27,2:9]
print(img.shape)
plt.imshow(img, cmap="gray", vmin=0, vmax=1)fig, axes = plt.subplots(7,7, figsize=(10, 10))
for xi in range(7):
    for yi in range(7):
        ax = axes[xi][yi]
        ax.imshow(A[xi][yi], cmap="gray")
plt.show()

In [ ]:
X = 0.8 * X + 0.2 * (torch.softmax(X @ X.T, dim=-1) @ X)

In [ ]:
A = (X @ X.T).reshape((7,7,7,7))
#
fig, axes = plt.subplots(7,7, figsize=(10, 10))
for xi in range(7):
    for yi in range(7):
        ax = axes[xi][yi]
        ax.imshow(A[xi][yi], cmap="gray")
plt.show()

# Full img

In [ ]:
fig, axes = plt.subplots(7,7, figsize=(10, 10))x,_ = ds_train[0]
img = x[0]
print(img.shape)
plt.imshow(img, cmap="gray", vmin=0, vmax=1)

In [ ]:
pe = get_pe(28, 28)
E = torch.cat((img.unsqueeze(0), pe), dim=0)
E = E.permute(1,2,0).reshape(28*28, -1)

In [ ]:
A = torch.softmax(E @ E.T, dim=-1).reshape((28,28,28,28))

In [ ]:
fig, axes = plt.subplots(7,7, figsize=(15, 15))
for i in range(7):
    for j in range(7):
        ax = axes[i][j]
        a = A[i*4][j*4]
        ax.imshow(a, cmap="gray")

In [ ]:
E.shape

In [ ]:
ES = E[torch.randint(E.shape[0], (100,)),:]

In [ ]:
ESU = torch.softmax(ES @ ES.T,dim=-1) @ ES

In [ ]:
A = torch.softmax(E @ E.T, dim=-1fig, axes = plt.subplots(7,7, figsize=(15, 15))
for i in range(7):
    for j in range(7):
        ax = axes[i][j]
        a = A[i*4][j*4]
        ax.imshow(a, cmap="gray")).reshape((28,28,28,28))ESU.shape

# CIFAR

In [ ]:
ds = torchvision.datasets.CIFAR10(
    root='/mnt/data/pytorch', train=True, download=True, transform=T.ToTensor())

In [ ]:
x,_ = ds[100]
plt.imshow(x.permute(1,2,0))

In [ ]:
pe = get_pe(32, 32)
E = torch.cat((x, pe), dim=0)
#
E = E.permute(1,2,0).reshape(32*32, -1)
print(E.shape)

In [ ]:
A = E @ E.T
print(A.shape, A.sum(dim=1))
A = A.reshape((32,32,32,32))

In [ ]:
A.min(), A.max()

In [ ]:
fig, axes = plt.subplots(8,8, figsize=(15, 15))
for i in range(8):
    for j in range(8):
        ax = axes[i][j]
        a = A[i*4][j*4]
        a[i*4,j*4] = A.max()
        ax.imshow(a, cmap="gray", vmin=A.min(), vmax=A.max())
plt.show()